<a href="https://colab.research.google.com/github/AxithChoudhary/Chatbot-coincent/blob/main/CoinCent_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##ChatBot

In [1]:
import pickle
import os
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
with open("/content/drive/MyDrive/Dataset/Coincent NLP /dataset/train_qa.txt","rb") as fp:
    train_data=pickle.load(fp)
print(train_data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
with open("/content/drive/MyDrive/Dataset/Coincent NLP /dataset/test_qa.txt","rb") as fp:
    test_data=pickle.load(fp)
print(test_data)

[(['Mary', 'got', 'the', 'milk', 'there', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.'], ['Is', 'John', 'in', 'the', 'kitchen', '?'], 'no'), (['Mary', 'got', 'the', 'milk', 'there', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.', 'Mary', 'discarded', 'the', 'milk', '.', 'John', 'went', 'to', 'the', 'garden', '.'], ['Is', 'John', 'in', 'the', 'kitchen', '?'], 'no'), (['Mary', 'got', 'the', 'milk', 'there', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.', 'Mary', 'discarded', 'the', 'milk', '.', 'John', 'went', 'to', 'the', 'garden', '.', 'Daniel', 'moved', 'to', 'the', 'bedroom', '.', 'Daniel', 'went', 'to', 'the', 'garden', '.'], ['Is', 'John', 'in', 'the', 'garden', '?'], 'yes'), (['Mary', 'got', 'the', 'milk', 'there', '.', 'John', 'moved', 'to', 'the', 'bedroom', '.', 'Mary', 'discarded', 'the', 'milk', '.', 'John', 'went', 'to', 'the', 'garden', '.', 'Daniel', 'moved', 'to', 'the', 'bedroom', '.', 'Daniel', 'went', 'to', 'the', 'garden', '.', 'Daniel', 'travelled', 'to', 

In [4]:
train_data[0]

(['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'Sandra', 'in', 'the', 'hallway', '?'],
 'no')

In [5]:
print(f"story: {train_data[0][0]},question: {train_data[0][1]}, answer: {train_data[0][2]}")

story: ['Mary', 'moved', 'to', 'the', 'bathroom', '.', 'Sandra', 'journeyed', 'to', 'the', 'bedroom', '.'],question: ['Is', 'Sandra', 'in', 'the', 'hallway', '?'], answer: no


In [6]:
vocab=set()
all_data=test_data+train_data

In [7]:
for story,question,answer in all_data:
    vocab=vocab.union(set(story))
    vocab=vocab.union(set(question))
vocab.add("no")
vocab.add("yes")

In [8]:
vocab_size=len(vocab)+1

In [9]:
all_data[0]

(['Mary',
  'got',
  'the',
  'milk',
  'there',
  '.',
  'John',
  'moved',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Is', 'John', 'in', 'the', 'kitchen', '?'],
 'no')

In [10]:
max_story_len=max([len(story[0]) for story in all_data])
print(max_story_len)

156


In [11]:
max_question_len=max([len(story[1]) for story in all_data])
print(max_question_len)

6


In [12]:
tokenizer=Tokenizer(filters=[])
tokenizer.fit_on_texts(vocab)
tokenizer.word_index

{'milk': 1,
 'hallway': 2,
 'mary': 3,
 'moved': 4,
 'down': 5,
 'apple': 6,
 'went': 7,
 'got': 8,
 'travelled': 9,
 'grabbed': 10,
 'daniel': 11,
 'took': 12,
 'discarded': 13,
 'in': 14,
 'office': 15,
 'yes': 16,
 'to': 17,
 'kitchen': 18,
 'picked': 19,
 'dropped': 20,
 'the': 21,
 'there': 22,
 'left': 23,
 'bathroom': 24,
 'put': 25,
 'no': 26,
 'bedroom': 27,
 'back': 28,
 'sandra': 29,
 'football': 30,
 'john': 31,
 'journeyed': 32,
 'is': 33,
 '?': 34,
 '.': 35,
 'garden': 36,
 'up': 37}

In [13]:
train_story_text=[]
train_question_text=[]
train_answer=[]

for story,question,answer in train_data:
  train_story_text.append(story)
  train_question_text.append(question)

In [14]:
train_story_text

[['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.'],
 ['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.',
  'Mary',
  'went',
  'back',
  'to',
  'the',
  'bedroom',
  '.',
  'Daniel',
  'went',
  'back',
  'to',
  'the',
  'hallway',
  '.'],
 ['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.',
  'Mary',
  'went',
  'back',
  'to',
  'the',
  'bedroom',
  '.',
  'Daniel',
  'went',
  'back',
  'to',
  'the',
  'hallway',
  '.',
  'Sandra',
  'went',
  'to',
  'the',
  'kitchen',
  '.',
  'Daniel',
  'went',
  'back',
  'to',
  'the',
  'bathroom',
  '.'],
 ['Mary',
  'moved',
  'to',
  'the',
  'bathroom',
  '.',
  'Sandra',
  'journeyed',
  'to',
  'the',
  'bedroom',
  '.',
  'Mary',
  'went',
  'back',
  'to',
  'the',
  'bedroom',
  '.',
  'Daniel',
  'went',
  'back',
  'to

In [15]:
train_story_seq=tokenizer.texts_to_sequences(train_story_text)
len(train_story_seq)

10000

In [16]:
train_story_seq

[[3, 4, 17, 21, 24, 35, 29, 32, 17, 21, 27, 35],
 [3,
  4,
  17,
  21,
  24,
  35,
  29,
  32,
  17,
  21,
  27,
  35,
  3,
  7,
  28,
  17,
  21,
  27,
  35,
  11,
  7,
  28,
  17,
  21,
  2,
  35],
 [3,
  4,
  17,
  21,
  24,
  35,
  29,
  32,
  17,
  21,
  27,
  35,
  3,
  7,
  28,
  17,
  21,
  27,
  35,
  11,
  7,
  28,
  17,
  21,
  2,
  35,
  29,
  7,
  17,
  21,
  18,
  35,
  11,
  7,
  28,
  17,
  21,
  24,
  35],
 [3,
  4,
  17,
  21,
  24,
  35,
  29,
  32,
  17,
  21,
  27,
  35,
  3,
  7,
  28,
  17,
  21,
  27,
  35,
  11,
  7,
  28,
  17,
  21,
  2,
  35,
  29,
  7,
  17,
  21,
  18,
  35,
  11,
  7,
  28,
  17,
  21,
  24,
  35,
  11,
  19,
  37,
  21,
  30,
  22,
  35,
  11,
  7,
  17,
  21,
  27,
  35],
 [3,
  4,
  17,
  21,
  24,
  35,
  29,
  32,
  17,
  21,
  27,
  35,
  3,
  7,
  28,
  17,
  21,
  27,
  35,
  11,
  7,
  28,
  17,
  21,
  2,
  35,
  29,
  7,
  17,
  21,
  18,
  35,
  11,
  7,
  28,
  17,
  21,
  24,
  35,
  11,
  19,
  37,
  21,
  30,
  22,
  35,
 

In [17]:
tokenizer.word_index

{'milk': 1,
 'hallway': 2,
 'mary': 3,
 'moved': 4,
 'down': 5,
 'apple': 6,
 'went': 7,
 'got': 8,
 'travelled': 9,
 'grabbed': 10,
 'daniel': 11,
 'took': 12,
 'discarded': 13,
 'in': 14,
 'office': 15,
 'yes': 16,
 'to': 17,
 'kitchen': 18,
 'picked': 19,
 'dropped': 20,
 'the': 21,
 'there': 22,
 'left': 23,
 'bathroom': 24,
 'put': 25,
 'no': 26,
 'bedroom': 27,
 'back': 28,
 'sandra': 29,
 'football': 30,
 'john': 31,
 'journeyed': 32,
 'is': 33,
 '?': 34,
 '.': 35,
 'garden': 36,
 'up': 37}

In [18]:
def vectorize_story(data,word_index=tokenizer.word_index
                    ,max_story_len=max_story_len
                    ,max_question_len=max_question_len):
  X=[]
  Xq=[]
  Y=[]


  #here we are assigning the number to each word in the sentence
  for story,question,answer in data:
    x=[word_index[word.lower()] for word in story]
    xq=[word_index[word.lower()] for word in question]
    y=np.zeros(len(word_index)+1)
    X.append(x)
    Xq.append(xq)
    Y.append(y)

  return(pad_sequences(X,max_story_len),pad_sequences(Xq,max_question_len),np.array(Y))



In [19]:
input_train,queries_train,answers_train=vectorize_story(train_data)
input_test,queries_test,answer_test=vectorize_story(test_data)
answer_test[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [20]:
[len(i) for i in answers_train]

[38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,
 38,


In [21]:
queries_train

array([[33, 29, 14, 21,  2, 34],
       [33, 11, 14, 21, 24, 34],
       [33, 11, 14, 21, 15, 34],
       ...,
       [33, 29, 14, 21,  2, 34],
       [33,  3, 14, 21, 18, 34],
       [33,  3, 14, 21, 27, 34]], dtype=int32)

In [22]:
from keras.models import Sequential,Model
from keras.layers import Embedding
from keras.layers import Input,Dense,Activation,Permute,Dropout,add,dot,concatenate,LSTM



In [23]:
input_sequence=Input((max_story_len,))
question=Input((max_question_len,))

input_encoder_m=Sequential()
input_encoder_m.add(Embedding(input_dim=vocab_size,output_dim=64))
input_encoder_m.add(Dropout(0.3))

input_encoder_c=Sequential()
input_encoder_c.add(Embedding(input_dim=vocab_size,output_dim=max_question_len))
input_encoder_c.add(Dropout(0.3))

question_encoder=Sequential()
question_encoder.add(Embedding(input_dim=vocab_size,output_dim=64,input_length=max_question_len))
question_encoder.add(Dropout(0.3))

input_encoded_m=input_encoder_m(input_sequence)
input_encoded_c=input_encoder_c(input_sequence)
question_encoded=question_encoder(question)

match=dot([input_encoded_m,question_encoded],axes=(2,2))
match=Activation("softmax")(match)

response=add([match,input_encoded_c])
response=Permute((2,1))(response)

answer=concatenate([response,question_encoded])
answer

answer=LSTM(32)(answer)

In [24]:
answer=Dropout(0.5)(answer)
answer=Dense(vocab_size)(answer)
answer=Activation("softmax")(answer)
model=Model([input_sequence,question],answer)
model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 156)]        0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 6)]          0           []                               
                                                                                                  
 sequential (Sequential)        (None, None, 64)     2432        ['input_1[0][0]']                
                                                                                                  
 sequential_2 (Sequential)      (None, 6, 64)        2432        ['input_2[0][0]']                
                                                                                              

In [25]:
input_train

array([[ 0,  0,  0, ..., 21, 27, 35],
       [ 0,  0,  0, ..., 21,  2, 35],
       [ 0,  0,  0, ..., 21, 24, 35],
       ...,
       [ 0,  0,  0, ..., 21, 27, 35],
       [ 0,  0,  0, ...,  1, 22, 35],
       [ 0,  0,  0, ...,  6, 22, 35]], dtype=int32)

In [26]:
history=model.fit([input_train,queries_train],
                  answers_train,
                  batch_size=32,
                  epochs=5,
                  validation_data=([input_test,queries_test],answer_test))

Epoch 1/5
313/313 [==============================] - 10s 15ms/step - loss: 0.0000e+00 - accuracy: 0.0025 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
313/313 [==============================] - 3s 9ms/step - loss: 0.0000e+00 - accuracy: 0.0021 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 3/5
313/313 [==============================] - 2s 7ms/step - loss: 0.0000e+00 - accuracy: 0.0015 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 4/5
313/313 [==============================] - 3s 9ms/step - loss: 0.0000e+00 - accuracy: 0.0017 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 5/5
313/313 [==============================] - 3s 11ms/step - loss: 0.0000e+00 - accuracy: 9.0000e-04 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
